In [2]:
import numpy as np
import pandas as pd
import json
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup as bs
import lxml
import requests

Solving environment: done

# All requested packages already installed.



In [7]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
Toronta_page=requests.get(wikipedia_link)#hint: requests.get()
page=Toronta_page.text


In [8]:
soup=bs(page,'lxml')
tabl_tor=soup.find('div',class_="mw-parser-output").table
temp=[] 
i=0
for tbl_head in tabl_tor.find_all('th'):
    tt=tbl_head.text
    temp.append(tt)
temp[-1] = temp[-1].strip()
t_head=tuple(temp)
i=0
data=[]
txt=[]
for j in tabl_tor.find_all('td'):  
    tc=j.text
    data.append(tc)
    data[-1] = data[-1].strip()
    r1=[]
r2=[]
r3=[]
i=0
j=1
k=2
l=len(data)
for i in range(0,l,3):
    r1.append(data[i])
for j in range(1,l,3):
    r2.append(data[j])
for k in range(2,l,3):
    r3.append(data[k])
tbl={t_head[0]:r1,t_head[1]:r2,t_head[2]:r3}
df=pd.DataFrame(tbl)


In [9]:
df1=df[df.Borough != 'Not assigned']
df1.reset_index(drop=True, inplace=True)
L=len(df1)
for i in range(0,L):
    if ((df1.Neighbourhood[i]) == 'Not assigned'):
        df1.Neighbourhood[i]=df1.Borough[i]
df2=df1.groupby(['Postcode','Borough']).Neighbourhood.unique().reset_index()


/home/jupyterlab/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
Geo_data= pd.read_csv('Geospatial_data.csv')
Geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
Geo_data.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
Geo_data.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
neighborhoods=pd.merge(df2, Geo_data, on='Postcode', how='inner')
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [23]:
col=neighborhoods.columns
print(col)

Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


In [20]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [25]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [32]:
Scarborogh_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborogh_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476


In [33]:
address = 'Scarborough, TO'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 43.773077, -79.257774.


In [29]:
CLIENT_ID = '30EXXJY2HBSDPTTTYZFRPZAILSBAMOO0P4KGLCQWEZ3I1I4I' # your Foursquare ID
CLIENT_SECRET = 'ZRKNTYYRQSUX0Q4DDTYBRYYPUMOMD3VVSQYWZT2YELF3SNRX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 30EXXJY2HBSDPTTTYZFRPZAILSBAMOO0P4KGLCQWEZ3I1I4I
CLIENT_SECRET:ZRKNTYYRQSUX0Q4DDTYBRYYPUMOMD3VVSQYWZT2YELF3SNRX


In [35]:
neighborhood_latitude = Scarborogh_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborogh_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborogh_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of ['Rouge' 'Malvern'] are 43.806686299999996, -79.19435340000001.


In [43]:
Scarborogh_data.loc[1, 'Neighbourhood']

array(['Highland Creek', 'Rouge Hill', 'Port Union'], dtype=object)

In [44]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=30EXXJY2HBSDPTTTYZFRPZAILSBAMOO0P4KGLCQWEZ3I1I4I&client_secret=ZRKNTYYRQSUX0Q4DDTYBRYYPUMOMD3VVSQYWZT2YELF3SNRX&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [45]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c121fea1ed2195d6ae58c05'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [46]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [47]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [48]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [49]:
# one hot encoding
Scarborogh_onehot = pd.get_dummies(Scarborogh_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborogh_onehot['Neighbourhood'] = Scarborogh_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborogh_onehot.columns[-1]] + list(Scarborogh_onehot.columns[:-1])
Scarborogh_onehot = Scarborogh_onehot[fixed_columns]

Scarborogh_onehot.head()

NameError: name 'Scarborogh_venues' is not defined

In [ ]:
# set number of clusters
kclusters = 5

Scarborogh_grouped_clustering = Scarborogh_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 